In [1]:
import pandas as pd
import numpy as np
# from numba import njit

import matplotlib.pyplot as plt
from matplotlib import cm
from numpy import linalg as LA
from tqdm import tqdm

# from sklearn import svm

from kernel_functions import * # gram_phi, count_kuplet_k, count_kuplet_3
from preprocessing import preprocessing

In [2]:
# load all data as the numpy array type
#X = pd.read_csv('data/Xtr1_mat50.csv', sep=' ', header=None).values
X_raw0 = pd.read_csv('data/Xtr0.csv', sep= ' ', header = None).values.reshape((-1))
X_raw1 = pd.read_csv('data/Xtr1.csv', sep=' ', header=None).values.reshape((-1))
X_raw2 = pd.read_csv('data/Xtr2.csv', sep=' ', header=None).values.reshape((-1))

# transform to an array of string
X_valid0 = pd.read_csv('data/Xte0.csv', sep=' ', header=None).values.reshape((-1))
X_valid1 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))
X_valid2 = pd.read_csv('data/Xte1.csv', sep=' ', header=None).values.reshape((-1))


Y0 = pd.read_csv('data/Ytr0.csv', sep=',', header=0)['Bound'].values
Y1 = pd.read_csv('data/Ytr1.csv', sep=',', header=0)['Bound'].values
Y2 = pd.read_csv('data/Ytr2.csv', sep=',', header=0)['Bound'].values

#print('numerical features shape', X.shape)
#print('numerical features first row', X[0])
print('sequences shape: ', X_raw0.shape)
print('sequence first row: ', X_raw0[0])
print('labels shape', Y0.shape)

('sequences shape: ', (2000,))
('sequence first row: ', 'TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG')
('labels shape', (2000,))


In [102]:
from kernel import *
import multiprocessing # import Pool


def solve_svm_kernel(X_train, X_test, Y_train, Y_test, kernel='k_gram_gaussian', k=3, lamb=0.1, gamma=0.1, kktreg=1e-9):
    """
    kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement']
    """
    assert kernel in ['k_gram', 'k_gram_gaussian', 'k_substring', 'local_alignement']
    
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if k == 3:
            X_train_process = np.array([count_kuplet_3(x) for x in X_train])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        else:
            X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train])
            X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
        
    if kernel=="k_gram":
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
        print(K_test.shape)
    
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        print(K_test.shape)
        K_test = np.exp(-K_test/gamma)
    
    if kernel=='k_substring':
        # Computing the Gram-Matrix
        # X_train_process = X_train
        # X_test_process = X_test
        N_train = len(X_train)
        N_test = len(X_test)
        K = np.zeros((N_train, N_train))
        for i in tqdm(range(len(X_train))):# ,desc="Computing Traning Kernel":
            # print("building pool")
            # pool = multiprocessing.Pool(4)
            # print("build param")
            # param = [(X_train[i], X_train[j], k) for j in range(len(X_train))]
            # print("using pool")
            # K[i][:] = pool.map(compute_gap_kernel_param, [[X_train[i], X_train[j], k] for j in range(len(X_train))])
            # print(K_test[i][:])
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = compute_gap_kernel(X_train[i], X_train[j], k)
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = compute_gap_kernel(X_test[i], X_train[j], k)
    
    if kernel=='local_alignement':
        # Computing the Gram-Matrix
        # X_train_process = X_train
        # X_test_process = X_test
        N_train = len(X_train)
        N_test = len(X_test)
        K = np.zeros((N_train, N_train))
        for i in tqdm(range(len(X_train))):# ,desc="Computing Traning Kernel":
            # print("building pool")
            # pool = multiprocessing.Pool(4)
            # print("build param")
            # param = [(X_train[i], X_train[j], k) for j in range(len(X_train))]
            # print("using pool")
            # K[i][:] = pool.map(compute_gap_kernel_param, [[X_train[i], X_train[j], k] for j in range(len(X_train))])
            # print(K_test[i][:])
            for j in range(i, len(X_train)):
                K[i][j] = K[j][i] = LA_kernel(X_train[i], X_train[j])
        w = solve_svm(K, Y_train, lamb=lamb, kktreg=kktreg)
        K_test = np.zeros((N_test, N_train))
        for i in tqdm(range(len(X_test)), desc="Compution testing Kernel"):
            for j in range(len(X_train)):
                K_test[i][j] = LA_kernel(X_test[i], X_train[j])
                
    # if kernel in ['k_gram', 'k_gram_gaussian']:
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    result = ((Y_test+1.)/ 2. == np.transpose(Y_predicted))
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = ((Y_train+1)/ 2 == np.transpose(Y_predicted_train))
    print(Y_predicted)
    if np.alltrue(Y_predicted):
        print("Toute les valeurs sont TRUE")
    if np.alltrue(Y_predicted==False):
        print("Toute les valeurs sont FALSE")
    return np.mean(result), np.mean(result_train)
             


In [4]:
X_train, Y_train, X_test, Y_test = preprocessing(X_raw1, Y1)
print('train shape', X_train.shape)
print('test shape', X_test.shape)

('train shape', (1600,))
('test shape', (400,))


In [5]:
X_train[:][0]

'GAGGGCCAGCTCCCGCACTAACAGAGTTGCTCTGCAGGGATGGTGCATAGCACAAAGAAAAGGTGATTTTGGCTGCAGAATTGAAAAACCATATGTATGGT'

In [7]:
def compute_gap_kernel(X1, X2, k, lamb=0.5):
    N_X1 = len(X1)
    N_X2 = len(X2)
    B = np.zeros((k+1, N_X1+1, N_X2+1))
    K = np.zeros((N_X1+1, N_X2+1))
    ker = np.zeros(k+1)
    for i in range(1, N_X1+1):
        for j in range(1, N_X2+1):
            if(X1[i-1] == X2[j-1]):
                B[1, i, j] = lamb**2 
                ker[1] += lamb**2 
    # ker = ker/(lamb**2) # renormalize
    for l in range(2, k+1):
        for i in range(1, N_X1+1):
            for j in range(1, N_X2+1):
                K[i, j] = B[l-1, i, j] + lamb*K[i-1, j] + lamb*K[i, j-1] - (lamb**2)*K[i-1, j-1]
                if X1[i-1]==X2[j-1]:
                    B[l, i, j] = lamb**2 * K[i-1, j-1]
                    ker[l] = ker[l] + B[l, i, j]
    return ker[k]

def compute_gap_kernel_param(param):
    return compute_gap_kernel(param[0], param[1], param[2])

X1 = 'CGGGGCA'
X2 = 'CCAT'
k = 3
ker = compute_gap_kernel(X1, X2, k, lamb=0.5)
# ker, B, K = compute_gap_kernel(X1, X2, 3, lamb=0.5)
print(ker)

0.0009765625


In [15]:
# With Cross validation

X_train0, Y_train0, X_test0, Y_test0 = preprocessing(X_raw0, Y0)
X_train1, Y_train1, X_test1, Y_test1 = preprocessing(X_raw1, Y1)
X_train2, Y_train2, X_test2, Y_test2 = preprocessing(X_raw2, Y2)
print('train shape', X_train0.shape)
print('test shape', X_test0.shape)

print('train shape', X_train1.shape)
print('test shape', X_test1.shape)

print('train shape', X_train2.shape)
print('test shape', X_test2.shape)


('train shape', (1600,))
('test shape', (400,))
('train shape', (1600,))
('test shape', (400,))
('train shape', (1600,))
('test shape', (400,))


In [103]:
acc_test0, acc_train0 = solve_svm_kernel(X_train0[:50], 
                                       X_test0[:10], 
                                       Y_train0[:50], 
                                       Y_test0[:10], 
                                       kernel='local_alignement', 
                                       k=3, lamb=0.001, gamma=5)

print('accuracy for train : {}'.format(acc_train0))
print('accuracy for test : {}'.format(acc_test0))

  6%|▌         | 3/50 [00:19<05:11,  6.62s/it]

KeyboardInterrupt: 

In [9]:
acc_test1, acc_train1 = solve_svm_kernel(X_train1, 
                                       X_test1, 
                                       Y_train1, 
                                       Y_test1, 
                                       kernel='k_gram', 
                                       k=3, lamb=0.01, gamma=1.)

print('accuracy for train : {}'.format(acc_train1))
print('accuracy for test : {}'.format(acc_test1))

     pcost       dcost       gap    pres   dres
 0:  3.3586e-01  5.1306e+02  1e+04  3e+00  1e+06
 1:  2.4435e+00 -1.5721e+02  2e+02  3e-02  2e+04
 2:  2.3893e+00 -6.4279e+00  9e+00  2e-03  7e+02
 3:  1.7351e+00 -2.8486e-01  2e+00  2e-04  9e+01
 4:  6.6843e-01  4.2466e-01  2e-01  1e-05  5e+00
 5:  5.6983e-01  4.9009e-01  8e-02  3e-06  1e+00
 6:  5.4689e-01  5.0801e-01  4e-02  1e-06  6e-01
 7:  5.3331e-01  5.1829e-01  2e-02  3e-07  2e-01
 8:  5.2768e-01  5.2256e-01  5e-03  9e-08  4e-02
 9:  5.2586e-01  5.2402e-01  2e-03  3e-08  1e-02
10:  5.2510e-01  5.2461e-01  5e-04  1e-09  7e-04
11:  5.2492e-01  5.2477e-01  2e-04  9e-10  1e-04
12:  5.2485e-01  5.2483e-01  1e-05  1e-09  9e-06
13:  5.2484e-01  5.2484e-01  3e-07  1e-09  2e-07
14:  5.2484e-01  5.2484e-01  6e-09  1e-09  3e-09
Optimal solution found.
accuracy for train : 0.7875
accuracy for test : 0.74


In [31]:
def edit_distance(str1, str2):
    """
    Compute the Edit Distance between str1 and str2.
    Return the number of insertion/deletion/substitution
    """
    n = len(str1)
    m = len(str2)
    table_str = np.zeros((n+1,m+1))

    table_str[0, :]=np.arange(m+1)
    table_str[:, 0] = np.arange(n+1)

    for i in range(1, n+1):
        for j in range(1, m+1):
            if(str1[i-1] == str2[j-1]):
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]) 
            else:
                table_str[i, j] = min(table_str[i-1, j] + 1, table_str[i, j-1] + 1, table_str[i-1, j-1]+2) 
                
    return table_str

X1 = 'ACCCTG'
X2 = 'AGGTC'
print(edit_distance(X1, X2))

[[ 0.  1.  2.  3.  4.  5.]
 [ 1.  0.  1.  2.  3.  4.]
 [ 2.  1.  2.  3.  4.  3.]
 [ 3.  2.  3.  4.  5.  4.]
 [ 4.  3.  4.  5.  6.  5.]
 [ 5.  4.  5.  6.  5.  6.]
 [ 6.  5.  4.  5.  6.  7.]]


In [42]:
acc_test2, acc_train2 = solve_svm_kernel(X_train2, 
                                       X_test2, 
                                       Y_train2, 
                                       Y_test2, 
                                       kernel='k_gram_gaussian', 
                                       k=6, lamb=0.00001, gamma=1.)

print('accuracy for train : {}'.format(acc_train2))
print('accuracy for test : {}'.format(acc_test2))

     pcost       dcost       gap    pres   dres
 0:  1.5414e-02  1.6000e-02  1e+00  1e+00  4e-08
 1:  1.5994e-02  1.6000e-02  1e-02  1e-02  4e-10
 2:  1.6000e-02  1.6000e-02  1e-04  1e-04  3e-12
 3:  1.6000e-02  1.6000e-02  1e-06  1e-06  1e-13
 4:  1.6000e-02  1.6000e-02  1e-08  1e-08  2e-15
Optimal solution found.
accuracy for train : 1.0
accuracy for test : 0.5925


## Best parameters found for k_gram kernel
* Set0: lambda=1e-3, gamma = 5 (gaussian)
* Set1: lambda=0.01
* Set2: lambda= 1e-5

In [17]:
def solve_svm_test(X_train, X_test, Y_train, kernel='k_gram_gaussian', k=3, lamb=0.1, gamma=0.1, kktreg=1e-9):
    """
    kernel in [k_gram, k_gram_gaussian]
    """
    Y_train_process = (Y_train-0.5) * 2
    if kernel in ['k_gram', 'k_gram_gaussian']:
        if k == 3:
            X_train_process = np.array([count_kuplet_3(x) for x in X_train])
            X_test_process = np.array([count_kuplet_3(x) for x in X_test])
        else:
            X_train_process = np.array([count_kuplet_k(x,k=k) for x in X_train])
            X_test_process = np.array([count_kuplet_k(x,k=k) for x in X_test])
        
        # Adding 1 for the sake of the bias
        X_train_process = np.concatenate((X_train_process, np.ones((X_train_process.shape[0], 1))), axis=1)
        X_test_process = np.concatenate((X_test_process, np.ones((X_test_process.shape[0], 1))), axis=1)
        
        
    if kernel=="k_gram":
        # Computing the Gram-Matrix
        K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb, kktreg=kktreg)
        K_test = np.dot(X_test_process, np.transpose(X_train_process))
    
    if kernel=="k_gram_gaussian":
        # Computing the Gram-Matrix
        K = np.array([LA.norm(X_train_process - y, axis=1) for y in X_train_process])
        K = np.exp(-K/gamma)
        # K = X_train_process.dot(X_train_process.T)
        w = solve_svm(K, Y_train_process, lamb=lamb)
        K_test = np.array([LA.norm(X_train_process - y, axis=1) for y in X_test_process])
        K_test = np.exp(-K_test/gamma)
    
    n = K.shape[0]
    Y_predicted = np.dot(K_test, w[:n]) > 0.
    Y_predicted = (Y_predicted + 0.)
    Y_predicted_train = np.dot(K, w[:n]) > 0.
    result_train = (Y_train == np.transpose(Y_predicted_train))
    print('accuracy on train : {}'.format(np.mean(result_train)))
    # result = ((Y_test+1)/ 2 == np.transpose(Y_predicted))
    
    return np.transpose(Y_predicted) # , np.transpose(Y_predicted_train)


In [18]:
print('train shape : {}'.format(X_raw0.shape))
print('train shape 2 : {}'.format(Y0.shape))
print('test shape : {}'.format(X_valid0.shape))

Y_predict0 = solve_svm_test(X_raw0, 
                           X_valid0, 
                           Y0, 
                           kernel='k_gram_gaussian', 
                           k=3, lamb=.001, gamma=5.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
     pcost       dcost       gap    pres   dres
 0:  1.9546e+00  2.9746e+00  4e+03  1e+00  2e+02
 1:  2.9161e+00 -5.7463e+01  6e+01  2e-02  2e+00
 2:  2.0943e+00 -9.6812e+00  1e+01  4e-03  4e-01
 3:  1.5247e+00  2.3207e-01  1e+00  2e-04  2e-02
 4:  8.5340e-01  7.5174e-01  1e-01  6e-07  7e-05
 5:  8.0636e-01  7.8541e-01  2e-02  1e-07  1e-05
 6:  7.9629e-01  7.9397e-01  2e-03  7e-09  8e-07
 7:  7.9516e-01  7.9504e-01  1e-04  6e-10  3e-08
 8:  7.9510e-01  7.9510e-01  3e-06  7e-10  6e-10
 9:  7.9510e-01  7.9510e-01  6e-08  7e-10  1e-11
Optimal solution found.
accuracy on train : 0.861


In [19]:
print('train shape : {}'.format(X_raw1.shape))
print('train shape 2 : {}'.format(Y1.shape))
print('test shape : {}'.format(X_valid1.shape))

Y_predict1 = solve_svm_test(X_raw1, 
                           X_valid1, 
                           Y1, 
                           kernel='k_gram', 
                           k=3, lamb=0.01, gamma=3.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
     pcost       dcost       gap    pres   dres
 0:  3.3956e-01  6.5897e+02  2e+04  3e+00  2e+06
 1:  2.5257e+00 -2.0302e+02  2e+02  3e-02  2e+04
 2:  2.4864e+00 -9.2458e+00  1e+01  2e-03  8e+02
 3:  1.9470e+00 -6.7863e-01  3e+00  2e-04  1e+02
 4:  7.4528e-01  4.4086e-01  3e-01  2e-07  1e-01
 5:  5.8324e-01  5.1002e-01  7e-02  5e-08  2e-02
 6:  5.6350e-01  5.2487e-01  4e-02  2e-08  1e-02
 7:  5.5168e-01  5.3334e-01  2e-02  8e-09  4e-03
 8:  5.4558e-01  5.3790e-01  8e-03  3e-09  2e-03
 9:  5.4284e-01  5.3994e-01  3e-03  9e-10  4e-04
10:  5.4164e-01  5.4090e-01  7e-04  1e-09  6e-05
11:  5.4137e-01  5.4113e-01  2e-04  1e-09  9e-06
12:  5.4127e-01  5.4122e-01  5e-05  1e-09  2e-06
13:  5.4125e-01  5.4124e-01  9e-06  1e-09  3e-07
14:  5.4124e-01  5.4124e-01  9e-07  1e-09  2e-08
15:  5.4124e-01  5.4124e-01  2e-08  1e-09  3e-10
Optimal solution found.
accuracy on train : 0.772


In [20]:
print('train shape : {}'.format(X_raw2.shape))
print('train shape 2 : {}'.format(Y2.shape))
print('test shape : {}'.format(X_valid2.shape))

Y_predict2 = solve_svm_test(X_raw2, 
                           X_valid2, 
                           Y2, 
                           kernel='k_gram', 
                           k=3, lamb=1e-5, gamma=3.)

train shape : (2000,)
train shape 2 : (2000,)
test shape : (1000,)
     pcost       dcost       gap    pres   dres
 0:  4.3222e-01  8.6743e+02  2e+04  3e+00  3e+05
 1:  2.5361e+00 -1.7225e+02  2e+02  3e-02  3e+03
 2:  2.5071e+00 -3.5663e+00  6e+00  7e-04  8e+01
 3:  1.5970e+00  4.2108e-01  1e+00  6e-05  6e+00
 4:  8.3534e-01  6.7856e-01  2e-01  8e-06  8e-01
 5:  7.7959e-01  7.3967e-01  4e-02  2e-06  2e-01
 6:  7.6658e-01  7.5337e-01  1e-02  4e-07  4e-02
 7:  7.6288e-01  7.5704e-01  6e-03  2e-07  2e-02
 8:  7.6102e-01  7.5887e-01  2e-03  4e-08  4e-03
 9:  7.6032e-01  7.5955e-01  8e-04  1e-08  1e-03
10:  7.6008e-01  7.5977e-01  3e-04  3e-09  3e-04
11:  7.5996e-01  7.5989e-01  7e-05  8e-10  6e-05
12:  7.5993e-01  7.5991e-01  2e-05  9e-10  1e-05
13:  7.5992e-01  7.5992e-01  5e-06  1e-09  1e-06
14:  7.5992e-01  7.5992e-01  5e-07  1e-09  1e-07
15:  7.5992e-01  7.5992e-01  5e-09  1e-09  2e-09
Optimal solution found.
accuracy on train : 0.665


In [40]:
test0 = Y_predict0[:][0]
test1 = Y_predict1[:][0]
test2 = Y_predict2[:][0]

bound = np.concatenate((test0,test1,test2), axis=1).reshape((-1)).astype(int)
final = pd.DataFrame(np.arange(3000), columns=['Id'])
final['Bound'] = bound
final.to_csv('result0.csv', index= None)

In [ ]:
# Check before submit if the final has good shape
final